# QuASAr Benchmark Sweeps

This notebook provides reusable helpers to generate QuASAr benchmark circuits and run sweeps across circuit sizes or depths. Use the cells below to configure experiments, execute QuASAr, compare baseline simulators, and explore ablation studies.

## Setup
Import the unified circuit generator together with the QuASAr planning and execution stack.

In [ ]:
from __future__ import annotations

import logging
import sys

import json
import sqlite3
from datetime import datetime
from pathlib import Path

from contextlib import contextmanager
from typing import Any, Callable, Dict, Iterable, List, Optional

try:
    import pandas as pd  # type: ignore
except Exception:  # pragma: no cover - optional dependency
    pd = None

try:
    from tqdm.auto import tqdm  # type: ignore
except Exception:  # pragma: no cover - optional dependency
    tqdm = None

from IPython.display import display

from benchmarks.unified import generate_benchmark_circuit
from quasar.analyzer import AnalysisResult, analyze
from quasar.gate_metrics import circuit_metrics
from quasar.planner import PlannerConfig, plan
from quasar.qusd import Plan, QuSD
from quasar.simulation_engine import ExecutionConfig, execute_plan
from quasar.baselines import run_single_baseline


## Helper utilities
The helper functions below orchestrate QuASAr runs, present tabular outputs, and configure ablation scenarios.

In [ ]:
BenchmarkRecord = Dict[str, Any]

LOGGER = logging.getLogger("quasar.benchmark_sweeps")
if not LOGGER.handlers:
    handler = logging.StreamHandler(sys.stdout)
    handler.setFormatter(logging.Formatter("[%(asctime)s] %(levelname)s %(message)s"))
    LOGGER.addHandler(handler)
LOGGER.propagate = False
LOGGER.setLevel(logging.INFO)



BENCHMARK_DB_PATH = Path("benchmarks.db")


def _json_default(value: Any):
    if isinstance(value, set):
        return list(value)
    if hasattr(value, "to_dict"):
        return value.to_dict()
    return str(value)


def _json_dumps(obj: Any) -> str:
    return json.dumps(obj, default=_json_default, sort_keys=True)


def ensure_benchmark_db(db_path: Path = BENCHMARK_DB_PATH) -> None:
    with sqlite3.connect(str(db_path)) as conn:
        conn.execute(
            """
            CREATE TABLE IF NOT EXISTS quasar_runs (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                created_at TEXT NOT NULL,
                family TEXT NOT NULL,
                varying TEXT NOT NULL,
                varying_value INTEGER NOT NULL,
                num_qubits INTEGER NOT NULL,
                depth INTEGER NOT NULL,
                estimated_cost REAL,
                num_qusds INTEGER,
                exec_wall_s REAL,
                peak_rss_bytes INTEGER,
                cache_hits INTEGER,
                cache_misses INTEGER,
                decision_trace TEXT,
                analysis_json TEXT NOT NULL,
                plan_json TEXT NOT NULL,
                execution_json TEXT NOT NULL,
                summary_json TEXT NOT NULL,
                circuit_kwargs_json TEXT,
                planner_overrides_json TEXT,
                execution_overrides_json TEXT,
                analysis_fn TEXT,
                force_full_search INTEGER NOT NULL
            )
            """
        )
        conn.execute(
            """
            CREATE TABLE IF NOT EXISTS baseline_runs (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                created_at TEXT NOT NULL,
                backend TEXT NOT NULL,
                family TEXT NOT NULL,
                varying TEXT NOT NULL,
                varying_value INTEGER NOT NULL,
                num_qubits INTEGER NOT NULL,
                depth INTEGER NOT NULL,
                ok INTEGER,
                elapsed_s REAL,
                wall_s_measured REAL,
                error TEXT,
                result_json TEXT NOT NULL,
                circuit_kwargs_json TEXT,
                baseline_kwargs_json TEXT
            )
            """
        )
        conn.commit()


def store_quasar_benchmark_run(
    *,
    summary: Dict[str, Any],
    outcome: Dict[str, Any],
    family: str,
    varying: str,
    varying_value: int,
    num_qubits: int,
    depth: int,
    circuit_kwargs: Optional[Dict[str, Any]],
    planner_overrides: Optional[Dict[str, Any]],
    execution_overrides: Optional[Dict[str, Any]],
    analysis_fn: Callable[[Any], AnalysisResult],
    force_full_search: bool,
    db_path: Path = BENCHMARK_DB_PATH,
) -> None:
    ensure_benchmark_db(db_path)
    payload = (
        datetime.utcnow().isoformat(timespec="seconds"),
        family,
        varying,
        int(varying_value),
        num_qubits,
        depth,
        summary.get("estimated_cost"),
        summary.get("num_qusds"),
        summary.get("exec_wall_s"),
        summary.get("peak_rss_bytes"),
        summary.get("cache_hits"),
        summary.get("cache_misses"),
        _json_dumps(summary.get("decision_trace", [])),
        _json_dumps(outcome.get("analysis", {})),
        _json_dumps(outcome.get("plan", {})),
        _json_dumps(outcome.get("execution", {})),
        _json_dumps(summary),
        _json_dumps(circuit_kwargs or {}),
        _json_dumps(planner_overrides or {}),
        _json_dumps(execution_overrides or {}),
        getattr(analysis_fn, "__name__", repr(analysis_fn)),
        int(bool(force_full_search)),
    )
    with sqlite3.connect(str(db_path)) as conn:
        conn.execute(
            """
            INSERT INTO quasar_runs (
                created_at,
                family,
                varying,
                varying_value,
                num_qubits,
                depth,
                estimated_cost,
                num_qusds,
                exec_wall_s,
                peak_rss_bytes,
                cache_hits,
                cache_misses,
                decision_trace,
                analysis_json,
                plan_json,
                execution_json,
                summary_json,
                circuit_kwargs_json,
                planner_overrides_json,
                execution_overrides_json,
                analysis_fn,
                force_full_search
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            payload,
        )
        conn.commit()


def store_baseline_benchmark_run(
    *,
    summary: Dict[str, Any],
    result: Dict[str, Any],
    backend: str,
    family: str,
    varying: str,
    varying_value: int,
    num_qubits: int,
    depth: int,
    circuit_kwargs: Optional[Dict[str, Any]],
    baseline_kwargs: Optional[Dict[str, Any]],
    db_path: Path = BENCHMARK_DB_PATH,
) -> None:
    ensure_benchmark_db(db_path)
    payload = (
        datetime.utcnow().isoformat(timespec="seconds"),
        backend,
        family,
        varying,
        int(varying_value),
        num_qubits,
        depth,
        summary.get("ok"),
        summary.get("elapsed_s"),
        summary.get("wall_s_measured"),
        summary.get("error"),
        _json_dumps(result),
        _json_dumps(circuit_kwargs or {}),
        _json_dumps(baseline_kwargs or {}),
    )
    with sqlite3.connect(str(db_path)) as conn:
        conn.execute(
            """
            INSERT INTO baseline_runs (
                created_at,
                backend,
                family,
                varying,
                varying_value,
                num_qubits,
                depth,
                ok,
                elapsed_s,
                wall_s_measured,
                error,
                result_json,
                circuit_kwargs_json,
                baseline_kwargs_json
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            payload,
        )
        conn.commit()


def _iter_with_progress(values: Iterable[int], *, description: str):
    if tqdm is None:
        return values
    return tqdm(values, desc=description)


def _to_dataframe(records: Iterable[BenchmarkRecord]):
    if pd is not None:
        return pd.DataFrame(list(records))
    return list(records)


@contextmanager
def disable_quick_path():
    import quasar.planner as planner_module

    original = planner_module._should_use_quick_path
    planner_module._should_use_quick_path = lambda plan_obj, cfg: False
    try:
        yield
    finally:
        planner_module._should_use_quick_path = original


def analyze_without_disjoint(circuit) -> AnalysisResult:
    metrics = circuit_metrics(circuit)
    plan = Plan(meta={"total_qubits": circuit.num_qubits, "components": 1})
    qusd = QuSD(
        id=0,
        qubits=list(range(circuit.num_qubits)),
        circuit=circuit,
        metrics=dict(metrics),
    )
    plan.add(qusd)
    return AnalysisResult(plan=plan, metrics_global=dict(metrics))


def run_quasar_pipeline(
    circuit,
    *,
    planner_overrides: Optional[Dict[str, Any]] = None,
    execution_overrides: Optional[Dict[str, Any]] = None,
    analysis_fn: Callable[[Any], AnalysisResult] = analyze,
    force_full_search: bool = False,
) -> Dict[str, Any]:
    analysis = analysis_fn(circuit)
    planner_cfg = PlannerConfig(**(planner_overrides or {}))

    exec_kwargs = dict(execution_overrides or {})
    exec_kwargs.setdefault("max_ram_gb", planner_cfg.max_ram_gb)
    exec_cfg = ExecutionConfig(**exec_kwargs)

    if force_full_search:
        with disable_quick_path():
            planned = plan(analysis.plan, planner_cfg)
    else:
        planned = plan(analysis.plan, planner_cfg)

    execution = execute_plan(planned, exec_cfg)
    plan_payload = planned.to_dict()
    return {
        "analysis": analysis.metrics_global,
        "plan": {
            "estimated_cost": plan_payload["estimated_cost"],
            "num_qusds": len(planned.qusds),
            "decision_trace": list(plan_payload.get("decision_trace", [])),
            "meta": dict(plan_payload.get("meta", {})),
            "qusds": list(plan_payload.get("qusds", [])),
        },
        "execution": execution,
    }


def summarize_run(outcome: Dict[str, Any]) -> Dict[str, Any]:
    analysis = outcome["analysis"]
    plan_meta = outcome["plan"]
    exec_meta = outcome["execution"]["meta"]
    return {
        "num_qubits": int(analysis.get("num_qubits", 0)),
        "num_gates": int(analysis.get("num_gates", 0)),
        "two_qubit_gates": int(analysis.get("two_qubit_gates", 0)),
        "estimated_cost": float(plan_meta["estimated_cost"]),
        "num_qusds": int(plan_meta.get("num_qusds", 0)),
        "decision_trace": plan_meta.get("decision_trace", []),
        "exec_wall_s": float(exec_meta.get("wall_elapsed_s", 0.0)),
        "peak_rss_bytes": exec_meta.get("peak_rss_bytes"),
        "cache_hits": int(exec_meta.get("cache_hits", 0)),
        "cache_misses": int(exec_meta.get("cache_misses", 0)),
    }


def sweep_quasar(
    *,
    family: str,
    varying: str,
    values: Iterable[int],
    fixed_qubits: int,
    fixed_depth: int,
    circuit_kwargs: Optional[Dict[str, Any]] = None,
    planner_overrides: Optional[Dict[str, Any]] = None,
    execution_overrides: Optional[Dict[str, Any]] = None,
    analysis_fn: Callable[[Any], AnalysisResult] = analyze,
    force_full_search: bool = False,
) -> List[BenchmarkRecord]:
    ensure_benchmark_db()
    records: List[BenchmarkRecord] = []
    description = f"QuASAr {family} sweep ({varying})"
    for value in _iter_with_progress(values, description=description):
        if varying == "qubits":
            num_qubits = int(value)
            depth = int(fixed_depth)
        elif varying == "depth":
            num_qubits = int(fixed_qubits)
            depth = int(value)
        else:
            raise ValueError("varying must be 'qubits' or 'depth'")
        LOGGER.info(
            "Running QuASAr sweep point %s=%s (qubits=%d, depth=%d)",
            varying,
            value,
            num_qubits,
            depth,
        )
        circuit = generate_benchmark_circuit(
            num_qubits=num_qubits,
            depth=depth,
            family=family,
            **(circuit_kwargs or {}),
        )
        outcome = run_quasar_pipeline(
            circuit,
            planner_overrides=planner_overrides,
            execution_overrides=execution_overrides,
            analysis_fn=analysis_fn,
            force_full_search=force_full_search,
        )
        summary = summarize_run(outcome)
        summary.update({"family": family, "depth": depth})
        records.append(summary)
        store_quasar_benchmark_run(
            summary=summary,
            outcome=outcome,
            family=family,
            varying=varying,
            varying_value=num_qubits if varying == "qubits" else depth,
            num_qubits=num_qubits,
            depth=depth,
            circuit_kwargs=circuit_kwargs,
            planner_overrides=planner_overrides,
            execution_overrides=execution_overrides,
            analysis_fn=analysis_fn,
            force_full_search=force_full_search,
        )
        wall = summary.get("exec_wall_s")
        if wall:
            LOGGER.info(
                "Completed QuASAr sweep point %s=%s in %.2fs",
                varying,
                value,
                wall,
            )
        else:
            LOGGER.info("Completed QuASAr sweep point %s=%s", varying, value)
    return records


def sweep_baseline(
    *,
    backend: str,
    family: str,
    varying: str,
    values: Iterable[int],
    fixed_qubits: int,
    fixed_depth: int,
    circuit_kwargs: Optional[Dict[str, Any]] = None,
    baseline_kwargs: Optional[Dict[str, Any]] = None,
) -> List[BenchmarkRecord]:
    ensure_benchmark_db()
    records: List[BenchmarkRecord] = []
    description = f"{backend} baseline sweep ({family}, {varying})"
    for value in _iter_with_progress(values, description=description):
        if varying == "qubits":
            num_qubits = int(value)
            depth = int(fixed_depth)
        elif varying == "depth":
            num_qubits = int(fixed_qubits)
            depth = int(value)
        else:
            raise ValueError("varying must be 'qubits' or 'depth'")
        LOGGER.info(
            "Running %s baseline sweep point %s=%s (qubits=%d, depth=%d)",
            backend,
            varying,
            value,
            num_qubits,
            depth,
        )
        circuit = generate_benchmark_circuit(
            num_qubits=num_qubits,
            depth=depth,
            family=family,
            **(circuit_kwargs or {}),
        )
        result = run_single_baseline(
            circuit,
            backend,
            **(baseline_kwargs or {}),
        )
        payload = result.get("result", result)
        summary = {
            "backend": backend,
            "family": family,
            "num_qubits": num_qubits,
            "depth": depth,
            "ok": payload.get("ok"),
            "elapsed_s": payload.get("elapsed_s"),
            "wall_s_measured": payload.get("wall_s_measured"),
            "error": payload.get("error"),
        }
        records.append(summary)
        store_baseline_benchmark_run(
            summary=summary,
            result=result,
            backend=backend,
            family=family,
            varying=varying,
            varying_value=num_qubits if varying == "qubits" else depth,
            num_qubits=num_qubits,
            depth=depth,
            circuit_kwargs=circuit_kwargs,
            baseline_kwargs=baseline_kwargs,
        )
        elapsed = summary.get("elapsed_s") or summary.get("wall_s_measured")
        if summary.get("ok") is False and summary.get("error"):
            LOGGER.warning(
                "%s baseline sweep point %s=%s failed: %s",
                backend,
                varying,
                value,
                summary.get("error"),
            )
        elif elapsed:
            LOGGER.info(
                "Completed %s baseline sweep point %s=%s in %.2fs",
                backend,
                varying,
                value,
                elapsed,
            )
        else:
            LOGGER.info("Completed %s baseline sweep point %s=%s", backend, varying, value)
    return records


## Default sweep configuration
Adjust the values in this cell to target specific circuit families, qubit counts, and depths.

In [ ]:
family = "mixed"
qubit_sweep = [6, 8, 10]
depth_sweep = [8, 12, 16]

# Circuit-generation options are passed to ``generate_benchmark_circuit``.
circuit_options = {"block_size": 4, "cutoff": 0.75, "tail_type": "sparse", "seed": 7}

# Planner overrides are forwarded to :class:`PlannerConfig`.
planner_options = {"max_ram_gb": 4.0, "prefer_dd": True}

# Baseline runner parameters, e.g. to cap RAM usage.
baseline_options = {"max_ram_gb": 4.0, "timeout_s": 30.0}

## QuASAr sweeps
Run QuASAr across the configured sweep dimensions.

In [ ]:
quasar_qubit_results = sweep_quasar(
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
)

quasar_depth_results = sweep_quasar(
    family=family,
    varying="depth",
    values=depth_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
)

display(_to_dataframe(quasar_qubit_results))
display(_to_dataframe(quasar_depth_results))

## Tableau baseline sweeps

In [ ]:
tableau_qubit_results = sweep_baseline(
    backend="tableau",
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    baseline_kwargs=baseline_options,
)

display(_to_dataframe(tableau_qubit_results))

## Statevector baseline sweeps

In [ ]:
statevector_depth_results = sweep_baseline(
    backend="sv",
    family=family,
    varying="depth",
    values=depth_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    baseline_kwargs=baseline_options,
)

display(_to_dataframe(statevector_depth_results))

## Decision-diagram baseline sweeps

In [ ]:
dd_qubit_results = sweep_baseline(
    backend="dd",
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    baseline_kwargs=baseline_options,
)

display(_to_dataframe(dd_qubit_results))

## Ablation: disable disjoint parallelisation

In [ ]:
ablation_disjoint = sweep_quasar(
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
    analysis_fn=analyze_without_disjoint,
)

display(_to_dataframe(ablation_disjoint))

## Ablation: disable method-based partitioning

In [ ]:
ablation_method = sweep_quasar(
    family=family,
    varying="depth",
    values=depth_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides={**planner_options, "hybrid_clifford_tail": False},
)

display(_to_dataframe(ablation_method))

## Ablation: disable quick-path method selection

In [ ]:
ablation_quick_path = sweep_quasar(
    family=family,
    varying="qubits",
    values=qubit_sweep,
    fixed_qubits=qubit_sweep[0],
    fixed_depth=depth_sweep[0],
    circuit_kwargs=circuit_options,
    planner_overrides=planner_options,
    force_full_search=True,
)

display(_to_dataframe(ablation_quick_path))